In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string

In [3]:
def findFiles(path): 
    return glob.glob(path)

In [4]:
print(findFiles('datasets/data/names/*.txt'))

['datasets/data/names/Arabic.txt', 'datasets/data/names/French.txt', 'datasets/data/names/German.txt', 'datasets/data/names/Japanese.txt', 'datasets/data/names/Korean.txt', 'datasets/data/names/Russian.txt', 'datasets/data/names/English.txt', 'datasets/data/names/Irish.txt', 'datasets/data/names/Scottish.txt', 'datasets/data/names/Portuguese.txt', 'datasets/data/names/Greek.txt', 'datasets/data/names/Chinese.txt', 'datasets/data/names/Dutch.txt', 'datasets/data/names/Czech.txt', 'datasets/data/names/Polish.txt', 'datasets/data/names/Vietnamese.txt', 'datasets/data/names/Spanish.txt', 'datasets/data/names/Italian.txt']


In [5]:
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

In [6]:
# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )


In [8]:
print(unicodeToAscii('Ślusàrski'))

Slusarski
